In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests

In [2]:
pd.set_option('display.max_columns', None)

## Global Food Prices Database (Cleaned)
Updated Global Market Food Prices with 1.048M rows

In [3]:
price_df = pd.read_csv('../raw_data/Global Food Prices Database/wfp_food_prices_database.csv')

/Users/plyck/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
price_df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN


In [5]:
price_df.isnull().sum()/len(price_df)

adm0_id               0.000000
adm0_name             0.000000
adm1_id               0.000000
adm1_name             0.302027
mkt_id                0.000000
mkt_name              0.000000
cm_id                 0.000000
cm_name               0.000000
cur_id                0.000000
cur_name              0.000000
pt_id                 0.000000
pt_name               0.000000
um_id                 0.000000
um_name               0.000000
mp_month              0.000000
mp_year               0.000000
mp_price              0.000000
mp_commoditysource    1.000000
dtype: float64

In [6]:
#dropping the mp_commoditysource because it's 100% Nan's

#price_df.drop(columns='mp_commoditysource', inplace=True)

In [7]:
price_df['adm1_name'].nunique()

613

In [8]:
price_df['adm1_name'].fillna("Unknown", inplace=True)

In [9]:
price_df['adm1_name'].nunique()

614

In [10]:
price_df.isnull().sum()/len(price_df)

adm0_id               0.0
adm0_name             0.0
adm1_id               0.0
adm1_name             0.0
mkt_id                0.0
mkt_name              0.0
cm_id                 0.0
cm_name               0.0
cur_id                0.0
cur_name              0.0
pt_id                 0.0
pt_name               0.0
um_id                 0.0
um_name               0.0
mp_month              0.0
mp_year               0.0
mp_price              0.0
mp_commoditysource    1.0
dtype: float64

In [11]:
# Introducing new column 'type' where the cm_name (commodity name) is split to make it easier to read by the nutrition API

In [12]:
price_df['type'] = price_df['cm_name'].apply(lambda x: x.split('-')[0])

In [13]:
price_df.head()

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource,type
0,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,1,2014,50.0,NaN,Bread
1,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,2,2014,50.0,NaN,Bread
2,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,3,2014,50.0,NaN,Bread
3,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,4,2014,50.0,NaN,Bread
4,1.0,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread - Retail,0.0,AFN,15,Retail,5,KG,5,2014,50.0,NaN,Bread


In [14]:
#Filtering for year 2020 only

In [15]:
price_2020_df = price_df[price_df['mp_year'] == 2020]

In [16]:
price_2020_df.shape

(165098, 19)

In [17]:
#Dropping unneccesary columns

In [18]:
price_2020_df.drop(columns=['adm0_id', 'adm1_id', 'mkt_id', 'cm_id', 'cur_id', 'cm_name', 'pt_id', 'mp_commoditysource','um_id'], inplace=True)

/Users/plyck/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [19]:
#Removing rows with other units than 'KG' and 'L'.

In [20]:
price_2020_df = price_2020_df.loc[price_2020_df['um_name'].isin(['KG','L'])]

In [24]:
price_2020_df['cur_name'].unique()

array(['AFN', 'AOA', 'ARS', 'AMD', 'BDT', 'INR', 'BYR', 'XOF', 'BTN',
       'BOB', 'BIF', 'KHR', 'CVE', 'XAF', 'COP', 'CDF', 'DJF', 'USD',
       'EGP', 'ETB', 'GMD', 'GHS', 'GNF', 'IDR', 'IRR', 'IQD', 'JPY',
       'JOD', 'KZT', 'KES', 'KGS', 'LAK', 'LBP', 'LRD', 'LYD', 'MWK',
       'SOS', 'MRO', 'MXN', 'MNT', 'MZN', 'MMK', 'NAD', 'NPR', 'NGN',
       'PKR', 'PYG', 'PEN', 'PHP', 'RWF', 'SLL', 'SSP', 'LKR', 'NIS',
       'SZL', 'SYP', 'TJS', 'THB', 'TRY', 'UGX', 'YER', 'ZMW'],
      dtype=object)

In [25]:
# Source Wiki: It was previously known as the new Israeli shekel and the non-official abbreviation of NIS.
# Need to change NIS to ILS (Israeli New Shekel)

In [26]:
price_2020_df['cur_name'] = price_2020_df['cur_name'].apply(lambda x: x.replace('NIS','ILS'))

In [27]:
currencies = price_2020_df['cur_name'].unique()

In [28]:
type(currencies)

numpy.ndarray

In [29]:
currencies = currencies.tolist()

In [30]:
currencies.sort()

In [31]:
len(currencies)

62

In [32]:
#SSP = South Sudanese pound
# This is not in our currency exchange API so must be removed and added later.

In [33]:
currencies.remove("SSP")

In [34]:
len(currencies)

61

In [35]:
#API call to get the conversion rates in the list currencies

In [36]:
api_key = 'e3479b3ac491d8699be3a12b7cf7e51d'

In [37]:
curr = ','.join(currencies)

In [38]:
response = requests.get(f'http://api.currencylayer.com/live?access_key={api_key}&currencies={curr}&format=1')

In [39]:
response.url

'http://api.currencylayer.com/live?access_key=e3479b3ac491d8699be3a12b7cf7e51d&currencies=AFN,AMD,AOA,ARS,BDT,BIF,BOB,BTN,BYR,CDF,COP,CVE,DJF,EGP,ETB,GHS,GMD,GNF,IDR,ILS,INR,IQD,IRR,JOD,JPY,KES,KGS,KHR,KZT,LAK,LBP,LKR,LRD,LYD,MMK,MNT,MRO,MWK,MXN,MZN,NAD,NGN,NPR,PEN,PHP,PKR,PYG,RWF,SLL,SOS,SYP,SZL,THB,TJS,TRY,UGX,USD,XAF,XOF,YER,ZMW&format=1'

In [40]:
response.json()

{'success': True,
 'terms': 'https://currencylayer.com/terms',
 'privacy': 'https://currencylayer.com/privacy',
 'timestamp': 1638951004,
 'source': 'USD',
 'quotes': {'USDAFN': 96.25256,
  'USDAMD': 491.785255,
  'USDAOA': 564.209003,
  'USDARS': 101.364698,
  'USDBDT': 85.609632,
  'USDBIF': 1988.457958,
  'USDBOB': 6.88221,
  'USDBTN': 75.266214,
  'USDBYR': 19600,
  'USDCDF': 2001.99979,
  'USDCOP': 3910.98,
  'USDCVE': 97.76566,
  'USDDJF': 177.677891,
  'USDEGP': 15.729398,
  'USDETB': 48.065551,
  'USDGHS': 6.113668,
  'USDGMD': 52.50026,
  'USDGNF': 9482.344389,
  'USDIDR': 14360.45,
  'USDILS': 3.11779,
  'USDINR': 75.539018,
  'USDIQD': 1456.183202,
  'USDIRR': 42249.999669,
  'USDJOD': 0.709198,
  'USDJPY': 113.4265,
  'USDKES': 112.839953,
  'USDKGS': 84.797101,
  'USDKHR': 4063.738309,
  'USDKZT': 436.408126,
  'USDLAK': 10905.995939,
  'USDLBP': 1509.30014,
  'USDLKR': 202.358104,
  'USDLRD': 141.850169,
  'USDLYD': 4.592539,
  'USDMMK': 1776.528423,
  'USDMNT': 2858.2186

In [41]:
conversion_rates = response.json()

In [42]:
conversion_rates = conversion_rates['quotes']

In [43]:
len(conversion_rates), len(currencies)

(61, 61)

In [44]:
# Making a dictionary of the conversion rates

In [45]:
dic_values = []
for x in conversion_rates.keys():
    dic_values.append(conversion_rates[x])

In [46]:
len(dic_values)

61

In [47]:
conversion_rates_dict = dict(zip(currencies,dic_values))

In [48]:
#SSP will be added as 1 USD = 130.26 SSP

In [49]:
conversion_rates_dict['SSP'] = 130.26

In [50]:
# Adding a new column to our data with the conversion rates

In [51]:
price_2020_df['conv_rate'] = list(map(lambda x: conversion_rates_dict[x] , price_2020_df['cur_name']))

In [52]:
price_2020_df['conv_rate'].nunique()

62

In [53]:
# Adding a new column to our data with the price in USD

In [54]:
price_2020_df["usd_rate"] = price_2020_df["mp_price"] / price_2020_df["conv_rate"] 

In [55]:
price_2020_df

,adm0_name,adm1_name,mkt_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price,type,conv_rate,usd_rate
69,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,50.0000,Bread,96.252560,0.519467
70,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,2,2020,50.0000,Bread,96.252560,0.519467
71,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,3,2020,50.0000,Bread,96.252560,0.519467
72,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,4,2020,50.0000,Bread,96.252560,0.519467
275,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,25.3750,Wheat,96.252560,0.263629
...,...,...,...,...,...,...,...,...,...,...,...,...
1556443,Zambia,Western,Kalabo,ZMW,Retail,KG,2,2020,6.4480,"Maize meal (white, breakfast)",17.416263,0.370229
1556444,Zambia,Western,Kalabo,ZMW,Retail,KG,3,2020,6.6320,"Maize meal (white, breakfast)",17.416263,0.380794
1556445,Zambia,Western,Kalabo,ZMW,Retail,KG,4,2020,6.4400,"Maize meal (white, breakfast)",17.416263,0.369769
1556446,Zambia,Western,Kalabo,ZMW,Retail,KG,5,2020,6.5168,"Maize meal (white, breakfast)",17.416263,0.374179


In [56]:
price_2020_df.shape

(128938, 12)

In [57]:
price_2020_df['mp_month'].unique()

array([1, 2, 3, 4, 5, 6, 7, 8])

In [61]:
list_of_foods = price_2020_df['type'].unique()

In [62]:
list_of_foods = list_of_foods.tolist()

In [65]:
list_of_foods.sort()

In [66]:
list_of_foods

['Apples ',
 'Apples (dried) ',
 'Apples (red) ',
 'Avocados ',
 'Bananas ',
 'Bananas (imported) ',
 'Bananas (local) ',
 'Bananas (medium size) ',
 'Barley ',
 'Beans ',
 'Beans (black) ',
 'Beans (black, Nayarit) ',
 'Beans (black, Veracruz) ',
 'Beans (black, Zacatecas) ',
 'Beans (black, imported) ',
 'Beans (butter) ',
 'Beans (catarino) ',
 'Beans (cranberry) ',
 'Beans (dry) ',
 'Beans (fava, dry) ',
 'Beans (green, fresh) ',
 'Beans (haricot) ',
 'Beans (kidney red) ',
 'Beans (magnum) ',
 'Beans (niebe) ',
 'Beans (niebe, white) ',
 'Beans (pod) ',
 'Beans (red) ',
 'Beans (red, fresh) ',
 'Beans (string) ',
 'Beans (sugar) ',
 'Beans (white) ',
 'Beans (yardlong, green) ',
 'Beans(mash) ',
 'Beetroots ',
 'Blackberry ',
 'Bread ',
 'Bread (common) ',
 'Bread (first grade flour) ',
 'Bread (high grade flour) ',
 'Bread (pita) ',
 'Bread (wheat) ',
 'Broccoli ',
 'Buckwheat ',
 'Buckwheat grits ',
 'Bulgur ',
 'Bulgur (brown) ',
 'Butter ',
 'Butter (cow milk) ',
 'Butter (goa

In [72]:
len(list_of_foods)

422

In [86]:
list_of_non_food = ['Charcoal ','Corn Soy Blend (CSB++, food aid) ','Cotton ', 'Dishwashing liquid ','Disinfecting solution ','Fuel (Super Petrol) ','Fuel (diesel) ',
 'Fuel (diesel, parallel market) ',
 'Fuel (kerosene) ',
 'Fuel (petrol','Handwash soap ','Laundry detergent ','Laundry soap ','Salt ','Salt (iodised) ','Shampoo ']

In [88]:
len(list_of_non_food)

16

In [89]:
for x in list_of_non_food:
        print(x)
        price_2020_df = price_2020_df.loc[price_2020_df["type"] != x]

Charcoal 
Corn Soy Blend (CSB++, food aid) 
Cotton 
Dishwashing liquid 
Disinfecting solution 
Fuel (Super Petrol) 
Fuel (diesel) 
Fuel (diesel, parallel market) 
Fuel (kerosene) 
Fuel (petrol
Handwash soap 
Laundry detergent 
Laundry soap 
Salt 
Salt (iodised) 
Shampoo 


In [90]:
price_2020_df['type'].nunique()

406

## Importing Adrians Dataset

In [91]:
food_types = pd.read_csv('../raw_data/Food Types/food_types_v1.csv')

In [92]:
food_types.head()

,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
1,Wheat,10.7,4.53,47.5,5.73,473,0.0,125,274
2,Rice (low quality),0.3,19.2,16.0,4.2,110,24.0,0.0,967
3,Wage (qualified labour),0.0,0.0,71.4,not found,5360.0,not found,not found,357
4,Fuel (diesel),0.84,0.0,8.86,7.59,36.0,not found,15.0,38.0


In [93]:
food_types['food_type'].nunique(), price_2020_df['type'].nunique()

(522, 406)

In [94]:
price_2020_df.shape

(119885, 12)

In [99]:
joined_df = pd.merge(price_2020_df, food_types, how='left', left_on = 'type', right_on = 'food_type')

In [102]:
joined_df.head()

,adm0_name,adm1_name,mkt_name,cur_name,pt_name,um_name,mp_month,mp_year,mp_price,type,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,50.000,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
1,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,2,2020,50.000,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
2,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,3,2020,50.000,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
3,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,4,2020,50.000,Bread,96.25256,0.519467,Bread,5.88,7.06,51.8,15.3,247,0.0,24.0,306
4,Afghanistan,Badakhshan,Fayzabad,AFN,Retail,KG,1,2020,25.375,Wheat,96.25256,0.263629,Wheat,10.7,4.53,47.5,5.73,473,0.0,125,274


In [104]:
joined_df['type'].nunique()

406

# Drop columns with XXX_id as they do not mean much for us

In [ ]:
#df_condensed = joined_df.drop(columns=['adm0_id', 'adm1_id', 'mkt_id', 'cm_id', 'cur_id', 'pt_id', 'um_id'])

In [ ]:
#df_condensed.head()

In [ ]:
#df_condensed['um_name'].unique()

In [ ]:
#Need to get rid of: Electricity (kWh), Milling cost (sorghum) - Retail (LCU/3.5kg), Dishwashing liquid - Retail (900 ML), Diapers - Retail (28 pcs), Shampoo - Retail (250 ML), Toothpaste - Retail (150 mL and 100 ML), Toothbrush - Retail (Brush), Toothbrush - Retail (4 pcs),     

In [ ]:
#Maybes: Straw - Retail (Heap), 

In [ ]:
#df_condensed[df_condensed['um_name'] == '28 pcs'].head()

In [ ]:
# Removing everything that has 'not found' in kcal column. Like Diapers

In [ ]:
#df_food_only = df_condensed.loc[df_condensed["kcal"] != 'not found']

In [ ]:
#df_food_only['um_name'].unique()

In [ ]:
#Converting kcal column from string to float

In [ ]:
#df_food_only['kcal'] = df_food_only['kcal'].astype(float)

In [ ]:
# Removing everything that has 0 kCal

In [ ]:
#df_food_only = df_food_only.loc[(df_food_only["kcal"] > 0.0) ]

In [ ]:
#df_food_only.head()

In [ ]:
#df_food_only['um_name'].unique()

In [ ]:
#df_food_only[df_food_only['um_name'] == '11 KG'].head()

In [ ]:
#test = df_food_only.loc[df_food_only["um_name"].isin(['KG','L'])]

In [ ]:
#test['cm_name'].nunique()